# Load in data

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
import scipy.stats as scp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import itertools
from scipy.signal import hilbert
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as scs
import json
import pickle
import plotly.graph_objects as go
from tqdm.auto import tqdm
from itertools import combinations
import statsmodels.formula.api as smf

sns.set(context='talk', style='white', font='Arial')

today = date.today().strftime('%Y%m%d')

project_dir = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/'
data_dir = project_dir + 'proc/group/parcel_timeseries/sub_ts/'
out_dir = project_dir + 'proc/group/RSA/ISPS/'
sample_file = project_dir + 'proc/group/datasets_info/sample_gord.32k_fs_LR.pscalar.nii'
atlas_file = project_dir + 'proc/null_lL_WG33/Gordon333_SeitzmanSubcortical.32k_fs_LR.dlabel.nii'
os.makedirs(out_dir,exist_ok=True)

ax0 = nib.load(sample_file).header.get_axis(0)
ax1 = nib.load(sample_file).header.get_axis(1)

TR = 0.8

# load timeseries data info
subinfo = pd.read_csv(project_dir + 'proc/group/datasets/firstleveldatalabels_withpub_thresh0.8_20220412.csv', index_col=0)

# get network labels
parcel_labels = nib.load(sample_file).header.get_axis(1).name
network_labels = []
for s in parcel_labels:
    b = s.split('_')
    if len(b)<2:
        network_labels.append(b[0])
    else:
        network_labels.append(b[1])
network_labels = np.array(network_labels)
network_names, network_sizes = np.unique(network_labels, return_counts=True)

subinfo = subinfo.drop(['set','cond'], axis=1)
subinfo = subinfo.drop_duplicates()

# set up functions

## processing

In [2]:
def compile_ts_data(subdf, movie, datadir, outfile):
    """
    combine data for each movie together into 1 file
    
    Parameters
    ----------
    subdf: DataFrame
        A dataframe with subject IDs as the index. Includes IDs for all usable data.
    movie: str
        Corresponds with the str for the movie content to concatenate (e.g., "DM" or "TP").
    datadir: folder path
        Path to folder with the subject timeseries ciftis.
    outfile: file path
        Path including filename to save the output data of shape Ntimepoints x Nparcels x Nsubjects.
    
    Returns
    -------
    data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
    """
    if not isinstance(subdf, pd.DataFrame):
        subdf = pd.read_csv(subdf, index_col=0)
    
    for sub in subdf.index:
        file = '{0}{1}_task-movie{2}_bold1_AP_Atlas_rescale_resid0.9_filt_gordonseitzman.32k_fs_LR.ptseries.nii'.format(datadir,sub, movie)
        if sub == subdf.index[0]:
            data = StandardScaler().fit_transform(nib.load(file).get_fdata())
            data = np.expand_dims(data, axis=2)
        else:
            t = StandardScaler().fit_transform(nib.load(file).get_fdata())
            t = np.expand_dims(t, axis=2)
            data = np.concatenate([data,t],axis=2)
    
    print('Compile data from {0} brain regions measured at {1} timepoints from {2} participants.'.format(data.shape[1],data.shape[0],data.shape[2]))
    np.save(outfile, data)
    return(data)


def compute_group_phase(group_ts_data, outfile):
    """
    convert parcel ts to standard units & compute phase angles for each parcel ts
    
    Parameters
    ----------
    group_ts_data: filepath OR numpy array
        File or numpy array with compiled timeseries data of shape Ntimepoints x Nparcels x Nsubjects 
    
    Returns
    -------
    group_phase_data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
    """
    
    if not isinstance(group_ts_data, np.ndarray):
        group_ts_data = np.load(group_ts_data)
    
    group_phase_data = np.zeros_like(group_ts_data)
    
    for a in range(0,group_ts_data.shape[1]):
        for b in range(0,group_ts_data.shape[2]):
            group_phase_data[:,a,b] = np.angle(hilbert(group_ts_data[:,a,b]), deg=False)
    
    np.save(outfile, group_phase_data)
    
    return(group_phase_data)


def compute_isps(group_phase_data, outprefix, savemean=True):
    """
    parcel-wise inter-subject phase synchrony- output pairwise IPS and mean global IPS
    
    Parameters
    ----------
    group_phase_data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
        
    Returns
    -------
    isps_data: numpy array
        intersubject phase synchrony data of shape Nparcels x Nsubjects x Nsubjects x Ntimepoints
    mean_isps_data: numpy array
        intersubject phase synchrony data, averaged across time, of shape Nparcels x Nsubjects x Nsubjects
        
    """
    if not isinstance(group_phase_data, np.ndarray):
        group_phase_data = np.load(group_phase_data)
    
    if os.path.isdir(outprefix):
        file_name = os.path.join(outprefix, 'isps_data.dat')
    else:
        file_name = outprefix + 'isps_data.dat'

    isps_data = np.memmap(file_name, dtype=np.float32, mode='w+',
                          shape=(group_phase_data.shape[1],group_phase_data.shape[2],
                                 group_phase_data.shape[2],
                                 group_phase_data.shape[0]))
    
    subs = range(0, group_phase_data.shape[2])
    for region in range(0, group_phase_data.shape[1]):
        combs = itertools.combinations(subs, 2)
        for c in combs:
            sub1 = group_phase_data[:, region, c[0]]
            sub2 = group_phase_data[:, region, c[1]]
            a = 1 - np.sin(np.abs(sub1 - sub2) / 2)
            isps_data[region,c[0],c[1],:] = a
            isps_data[region,c[1],c[0],:] = a
        
    if savemean:
        mask = np.tri(isps_data.shape[2], isps_data.shape[2], -1, dtype=int)
        mean_isps_data = np.mean(isps_data[:,mask==1,:], axis=1)
        if os.path.isdir(outprefix):
            mean_file_name = os.path.join(outprefix, 'mean_isps_data.npy')
        else:
            mean_file_name = outprefix + 'mean_isps_data.npy'
        
        np.save(mean_file_name, mean_isps_data.T)
    
        return(mean_isps_data, isps_data)
    else:
        return(isps_data)
    
    
def intersubject_distance(data, outfile_prefix):
    """
    Compute static pairwise intersubject similarity
    
    Parameters
    ----------
    data: numpy array
        1D array of subject data (i.e., each participant contributes exactly 1 measure)
    outfilename: str
        name to save distance data to
    
    Returns
    -------
    isdistances: numpy array
        intersubject distances in the shape of Nsubjects x Nsubjects x Nmetrics
    """
    subs = range(0,data.shape[0])


    # NN
    nn = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        nn[c[0],c[1]] = np.max(data) - abs(data[c[0]] - data[c[1]])
        nn[c[1],c[0]] = np.max(data) - abs(data[c[0]] - data[c[1]])
    np.save(outfile_prefix + '_NN.npy', nn)

    # AnnaK mean
    annakmean = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakmean[c[0],c[1]] = (data[c[0]] + data[c[1]]) / 2
        annakmean[c[1],c[0]] = (data[c[0]] + data[c[1]]) / 2
    np.save(outfile_prefix + '_annakmean.npy', annakmean)
    
    # AnnaK max min mean
    AnnaKmaxminmean = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        AnnaKmaxminmean[c[0],c[1]] = np.max(data) - ((data[c[0]] + data[c[1]]) / 2)
        AnnaKmaxminmean[c[1],c[0]] = np.max(data) - ((data[c[0]] + data[c[1]]) / 2)
    np.save(outfile_prefix + '_annakmaxminmean.npy', AnnaKmaxminmean)

    # AnnaK min
    annakmin = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakmin[c[0],c[1]] = min([data[c[0]],data[c[1]]])
        annakmin[c[1],c[0]] = min([data[c[0]],data[c[1]]])
    np.save(outfile_prefix + '_annakmin.npy', annakmin)

    # AnnaK max minus min
    annakmaxminmax = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakmaxminmax[c[0],c[1]] =np.max(data) -  max([data[c[0]],data[c[1]]])
        annakmaxminmax[c[1],c[0]] = np.max(data) - max([data[c[0]],data[c[1]]])
    np.save(outfile_prefix + '_annakmaxminmax.npy', annakmaxminmax)
        
    # AnnaK absmean
    annakabsmean = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakabsmean[c[0],c[1]] = abs(data[c[0]] - data[c[1]]) * ((data[c[0]] + data[c[1]]) / 2)
        annakabsmean[c[1],c[0]] = abs(data[c[0]] - data[c[1]]) * ((data[c[0]] + data[c[1]]) / 2)
    np.save(outfile_prefix + '_annakabsmean.npy', annakabsmean)
    
    isdistances = {'NN': nn, 
                   'AnnaKmean': annakmean, 
                   'AnnaKmin': annakmin, 
                   'AnnaKabsmean': annakabsmean, 
                   'AnnaKmaxminmean': AnnaKmaxminmean, 
                   'AnnaKmaxminmax': annakmaxminmax}
    return(isdistances)

## analysis

In [18]:
def synchrony_discrep_fdr_netlevel(disc_mean_sim, disc_null, rep_mean_sim, outprefix, video_dur, alpha=0.05, 
                                   TR=TR, parcel_labels=parcel_labels, atlas_file=atlas_file):
    # assign thresholds
    bon_alpha = np.sqrt(0.05/disc_mean_sim.shape[1])

    n = int(round((len(disc_null)+1)*bon_alpha,0))
    discbonmin = disc_null[len(disc_null)-n]
    n = int(round((len(rep_null)+1)*bon_alpha,0))
    repbonmin = rep_null[len(rep_null)-n]
    if discbonmin < np.percentile(disc_mean_sim, 1-alpha):
        discbonmin = np.percentile(disc_mean_sim, 1-alpha)
        repbonmin = np.percentile(rep_mean_sim, 1-alpha)
    print("Disc sig threshold: {0}".format(discbonmin))
    print("Rep sig threshold: {0}".format(repbonmin))

    # find sig rs that replicate
    disc_sig = (disc_mean_sim > discbonmin).astype(int)
    rep_sig = (rep_mean_sim > repbonmin).astype(int)

    sigrs = np.zeros(disc_sig.shape).astype(int)
    sigrs[(disc_sig==1) & (rep_sig==1)] = 1
    
    # find parcels with at least 5 consecutive seconds of sig synchrony
    sig_parcels = np.zeros(sigrs.shape[1]).astype(int)
    sigrs_df = pd.DataFrame(sigrs, columns=parcel_labels)
    sigrs_df['time'] = range(0, len(sigrs_df))
    for i, a in enumerate(parcel_labels):
        sigrs_df['segment'] = (sigrs_df[a].diff(1) != 0).astype(int).cumsum()
        dur = pd.DataFrame({'dur': sigrs_df.groupby('segment').time.last() - sigrs_df.groupby('segment').time.first(),
                            'value': sigrs_df.groupby('segment')[a].mean()}).reset_index(drop=True)
        dur = dur.loc[dur['value']==1]
        if dur['dur'].max() > 6:
            sig_parcels[i] = 1

    # average synchrony within networks and plot
    masked_disc = disc_mean_sim
    masked_disc[:,sig_parcels==0]=np.nan
    disc_sync_df = pd.DataFrame(disc_mean_sim, columns=network_labels)
    disc_sync_df = disc_sync_df.groupby(by=network_labels, axis=1).mean().dropna(axis=1)

    masked_rep = rep_mean_sim
    masked_rep[:,sig_parcels==0]=np.nan
    rep_sync_df = pd.DataFrame(rep_mean_sim, columns=network_labels)
    rep_sync_df = rep_sync_df.groupby(by=network_labels, axis=1).mean().dropna(axis=1)

    # make figs for sig parcels sorted by network/region group
    color = (142/255, 50/255, 209/255, 1)

    ax1 = nib.load(atlas_file).header.get_axis(1)
    data = nib.load(atlas_file).get_fdata()
    ax0 = nib.load(atlas_file).header.get_axis(0)
    newmap=dict()
    newmap[0] = ax0[0][1][0]
    for net in disc_sync_df:
        parcels_keep = parcel_labels[(network_labels==net) & (sig_parcels==1)]
        for a in range(1,len(parcel_labels) +1):
            if parcel_labels[a-1] in parcels_keep:
                newmap[a] = (parcel_labels[a-1], color)
            else:
                newmap[a] = (parcel_labels[a-1], (1,1,1,0))
        ax0.label[0] = newmap
        img = nib.cifti2.cifti2.Cifti2Image(data, (ax0, ax1))
        nib.save(img, outprefix + 'sig_averaged_parcels_{0}.dlabel.nii'.format(net))
        
    # Plot and analysis replicating peaks
    times = np.arange(0, video_dur, TR)

    results = dict()

    ### Make plots ###
    for net in disc_sync_df.columns:
        fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
        # plot discovery
        ax[0].plot(times, disc_sync_df[net], color='k')
        dcpeaks, dcproperties = scs.find_peaks(disc_sync_df[net], width=5, height=discbonmin)
        for i_c, c in enumerate(dcproperties['prominences']):
            ax[0].axvspan(dcproperties['left_ips'][i_c]*TR, dcproperties['right_ips'][i_c]*TR,
                        color='#BAB3BF', alpha=0.9)
        ax[0].set_xlim([0,video_dur])
        ax[0].set_title(net + ': Discovery', weight='bold')
        ax[0].set_ylabel('Synchrony')

        # plot replication
        ax[1].plot(times, rep_sync_df[net], color='k')
        rcpeaks, rcproperties = scs.find_peaks(rep_sync_df[net], width=5, height=repbonmin)
        for i_c, c in enumerate(rcproperties['prominences']):
            ax[1].axvspan(rcproperties['left_ips'][i_c]*TR, rcproperties['right_ips'][i_c]*TR,
                        color='#BAB3BF', alpha=0.9)
        ax[1].set_xlim([0,video_dur])
        ax[1].set_title(net + ': Replication', weight='bold')
        ax[1].set_ylabel('Synchrony')
        ax[1].set_xlabel('Time (s)')
        plt.tight_layout()
        #plt.show()
        plt.savefig(outprefix + 'peak_similarity_{0}.svg'.format(net))
        plt.close()

        # save results
        dcproperties = {k:v.tolist() for k,v in dcproperties.items()}
        rcproperties = {k:v.tolist() for k,v in rcproperties.items()}
        if (len(rcproperties['left_ips'])>0) & (len(dcproperties['left_ips'])>0):
            results[net] = {'Discovery': {'corr': dcproperties},
                               'Replication': {'corr': rcproperties}}


    ### Characterize differences in ratings for high versus low similarity points in the video ###

    # load ratings and shift forward 6 TRs (4.8 seconds)
    ratings = pd.read_csv(ratings_file, index_col=0)
    ratings = ratings.iloc[0:int(video_dur/TR),:]
    ratings.loc[:,ratings.columns] = MinMaxScaler().fit_transform(ratings.to_numpy())
    ratings.index = range(6,int(video_dur/TR)+6)
    ratings = pd.concat([ratings, pd.DataFrame(np.nan, index=np.arange(0,6,1), columns=ratings.columns)])
    ratings = ratings.sort_index()
    ratings = ratings.iloc[0:int(video_dur/TR),:]

    # identify areas within and outside the peaks
    for net in results.keys():
        discmask = np.zeros(times.shape)
        for i, c in enumerate(results[net]['Discovery']['corr']['peak_heights']):
            start = round(results[net]['Discovery']['corr']['left_ips'][i])
            end = round(results[net]['Discovery']['corr']['right_ips'][i])
            discmask[start:end] = 1

        repmask = np.zeros(times.shape)
        for i, c in enumerate(results[net]['Replication']['corr']['peak_heights']):
            start = round(results[net]['Replication']['corr']['left_ips'][i])
            end = round(results[net]['Replication']['corr']['right_ips'][i])
            repmask[start:end] = 1

        tmask = np.zeros(times.shape).astype(int)
        tmask[(discmask==1) & (repmask==1)] = 1

        ratesigdiff = pd.Series(index=ratings.columns, name='sig', dtype=int)
        ratemeans = dict()
        ratestats = dict()
        
        if tmask.max()==1:
            for measure in ratings.columns:
                underpeak = ratings.loc[tmask==1, measure]
                outofpeak = ratings.loc[tmask==0, measure]
                underpeakmean = np.nanmean(underpeak)
                outofpeakmean = np.nanmean(outofpeak)
                rate_stat, rate_pval = scp.ttest_ind(underpeak, outofpeak, nan_policy='omit')
                ratestats[measure] = {'tstat': rate_stat, 'pval': rate_pval}
                ratesigdiff[measure] = rate_pval < alpha
                ratemeans[measure] = {'underpeak': underpeakmean, 'outofpeak': outofpeakmean}

            results[net]['RatingsAnalysis'] = {'MeanRatings': ratemeans, 
                                                  'Stats': ratestats}

        # plot the differences
        sigratingsnames = ratesigdiff[ratesigdiff==True].index.to_list()

        if (len(sigratingsnames) > 0):
            ratings_withpeakinfo = ratings
            ratings_withpeakinfo['UnderPeak'] = tmask
            temp = pd.melt(ratings_withpeakinfo, id_vars='UnderPeak', value_vars=sigratingsnames, value_name='Level',var_name='Rating')
            plt.figure(figsize=(2+len(sigratingsnames),4))
            sns.barplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['#FFFFFF','#BCB1C2'], ci=None, edgecolor='k')
            sns.stripplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['k', 'k'], dodge=True, alpha=0.5)
            plt.legend([],[], frameon=False)
            plt.xticks(rotation=30, ha='right')
            plt.ylim(0,1.1)
            sns.despine()
            plt.tight_layout()
            plt.savefig(outprefix + 'peak_ratings_sigdiff_{0}.svg'.format(net))
            #plt.show()
            plt.close()
            ratings = ratings.drop('UnderPeak', axis=1)

        else:
            print('no overlap between discovery and replication for {0}'.format(net))

    #save results
    with open(outprefix + 'full_results_corrp{0}_prom{1}_width{2}.json'.format(round(bon_alpha,3), round(discbonmin,2), 5), 'w') as fp:
        json.dump(results, fp, indent=4)
        
    return(disc_sync_df, rep_sync_df)


def find_peak_rating_diffs(peak_mask, ratings_file, video_dur, out_file, color='#BAB3BF', TR=TR, alpha=0.05, fdr=True, shift=6):
    '''
    Parameters
    ----------
    peak_mask: numpy ndarray
        timeseries mask in the shape of Nsamples, with 1 indicating peak and 0 indiciating nonpeak.
    ratings_file: filename
        CSV file containing the ratings to use to chracterize peak versus non-peak
    video_dur: float
        duration in seconds of the movie
    out_file: string
        Name to save the plot under
    TR = float
        Repetition Time in seconds
    alpha: float
        p-value to determine significance for t-tests (peak versus nonpeak)
    shift: int
        Number of samples to shift the ratings over to account for the delayed hemodynamic response
        
    Returns
    -------
    results: dict
        dictionary with full t-test results across all ratings
    
    '''
    
    results={'MeanRatings':dict(), 'Stats':dict(), 'pvals': dict()}

    ratings = pd.read_csv(ratings_file, index_col=0)
    ratings = ratings.iloc[0:int(video_dur/TR),:]
    ratings.loc[:,ratings.columns] = MinMaxScaler().fit_transform(ratings.to_numpy())
    ratings.index = range(6,int(video_dur/TR)+6)
    ratings = pd.concat([ratings, pd.DataFrame(np.nan, index=np.arange(0,6,1), columns=ratings.columns)])
    ratings = ratings.sort_index()
    ratings = ratings.iloc[0:int(video_dur/TR),:]

    for measure in ratings.columns:
        underpeak = ratings.loc[peak_mask==1, measure]
        outofpeak = ratings.loc[peak_mask==0, measure]
        underpeakmean = np.nanmean(underpeak)
        outofpeakmean = np.nanmean(outofpeak)
        rate_stat, rate_pval = scp.ttest_ind(underpeak, outofpeak, nan_policy='omit')
        results['Stats'][measure] = {'tstat': rate_stat, 'pval': rate_pval}
        results['pvals'][measure] = rate_pval
        results['MeanRatings'][measure] = {'underpeak': underpeakmean, 'outofpeak': outofpeakmean}

    # plot the differences
    if not fdr:
        sigratingsnames = [k for (k, v) in results['pvals'].items() if v<alpha]
    else: 
        from statsmodels.stats.multitest import multipletests
        pvals = list(results['pvals'].values())
        meas_names = list(results['pvals'].keys())
        sig, q, _, _ = multipletests(pvals, alpha, method='fdr_bh')
        sigratingsnames = [m for i,m in enumerate(meas_names) if q[i]<alpha]

    if (len(sigratingsnames) > 0):
        ratings_withpeakinfo = ratings
        ratings_withpeakinfo['UnderPeak'] = peak_mask
        temp = pd.melt(ratings_withpeakinfo, id_vars='UnderPeak', value_vars=sigratingsnames, value_name='Level',var_name='Rating')
        plt.figure(figsize=(2+len(sigratingsnames),4))
        sns.barplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['#FFFFFF', color], ci=None, edgecolor='k')
        sns.stripplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['k', 'k'], dodge=True, alpha=0.5)
        plt.legend([],[], frameon=False)
        plt.ylim(0,1.1)
        plt.xticks(rotation=30, ha='right')
        sns.despine()
        plt.tight_layout()
        plt.savefig(out_file)
        #plt.show()
        plt.close()
        ratings = ratings.drop('UnderPeak', axis=1)
    else:
        print('No differences between peak and nonpeak ratings')
        
    return(results)


def match_peak_to_clips(peaktimes, shift, video_file, outfolder, ratings_file, bool_list=None, dim_list=None):
    '''
    
    Parameters
    ----------
    peaktimes: DataFrame
        DataFrame containing and index of unique values (will be how clips are named) and a 'start' and 'end' column 
        with times in seconds.
    shift: float
        Time in seconds to shift the peak onset/offset by (is subtracted from each)
    video_file: str
        the moviepy compatible video file to pull clips from
    outfolder: str
        the folder path to save the clips under
    ratings_file: str
        CSV containing a pandas dataframe with an index of time in seconds and columns of video features to plot
    '''
    
    import moviepy.video.io.ffmpeg_tools as mpff
    from sklearn.preprocessing import MinMaxScaler
    features = pd.read_csv(ratings_file, index_col=0)
    features.loc[:,:] = MinMaxScaler().fit_transform(features.to_numpy())

    for clipnum in peaktimes.index:
        outfile = os.path.join(outfolder, 'clip{0}.mp4'.format(clipnum))
        start = peaktimes.loc[clipnum,'start'] - shift - 4
        end = peaktimes.loc[clipnum,'end'] - shift + 4
        if start<0:
            start=0
        if end>0:
            mpff.ffmpeg_extract_subclip(video_file, start, end, outfile)
        if not bool_list and not dim_list:
            fig, ax = plt.subplots(figsize=(6,4))
            features.loc[start:end,:].plot(kind='line', ax=ax)
            plt.tight_layout()
            sns.despine()
            plt.savefig(os.path.join(outfolder, 'clip{0}.svg'.format(clipnum)))
            plt.close()
        else:
            try:
                fig, ax = plt.subplots(figsize=(8,1.5*len(bool_list)))
                ax = features.loc[start:end,bool_list].plot(subplots=True, kind='area', ax=ax, xlim=(start,end), sharex=True, sharey=True)
                for a in range(0, len(bool_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_boolvarsleg.svg'.format(clipnum)))
                plt.close()
                fig, ax = plt.subplots(figsize=(8,1.5*len(bool_list)))
                ax = features.loc[start:end,bool_list].plot(subplots=True, kind='area', ax=ax, legend=False, xlim=(start,end), sharex=True, sharey=True)
                for a in range(0, len(bool_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_boolvars_noleg.svg'.format(clipnum)))
                plt.close()
            except:
                pass
            try:
                fig, ax = plt.subplots(figsize=(6,1.5*len(dim_list)))
                ax = features.loc[start:end,dim_list].plot(subplots=True, ax=ax, xlim=(start,end), sharex=True)
                for a in range(0, len(dim_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_dimvarsleg.svg'.format(clipnum)))
                plt.close()
                fig, ax = plt.subplots(figsize=(6,1.5*len(dim_list)))
                ax = features.loc[start:end,dim_list].plot(subplots=True, ax=ax, legend=False, xlim=(start,end), sharex=True)
                for a in range(0, len(dim_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_dimvars_nleg.svg'.format(clipnum)))
                plt.close()
            except:
                pass
            
            
def plot_network_activation(ts_data, nois, network_labels, time, groups, group_labels, outdir):
    
    colors = ['b','k','r','g','y']
    # convert to percent signal change
    ts_psc_data = (ts_data - ts_data.min(axis=0, keepdims=True)) / (ts_data.max(axis=0, keepdims=True) - ts_data.min(axis=0, keepdims=True))
    mean_sig = np.mean(ts_psc_data, axis=0, keepdims=True)
    ts_psc_data = ((ts_psc_data-mean_sig)/mean_sig)*100

    # average across network/region
    ts_psc_net_data = np.zeros((ts_psc_data.shape[0], len(nois), ts_psc_data.shape[2]))
    for i, n in enumerate(nois):
        ts_psc_net_data[:,i,:] = np.mean(ts_psc_data[:,network_labels==n,:], axis=1)

    # plot group level traces
    fig, ax = plt.subplots(len(nois),1,figsize=(12,2*len(nois)), sharex=True)
    for i, g in enumerate(groups):
        mean_sig = np.mean(ts_psc_net_data[:,:,group_labels==g], axis=2)
        for j, net in enumerate(nois):  
            ax[j].plot(time, mean_sig[:,j], color=colors[i], label=g)
            ax[j].set_xlim((0,time[-1]))
            ax[j].set_title(net)

    sns.despine()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'network_activation_noleg.svg'))
    plt.legend()
    plt.savefig(os.path.join(outdir, 'network_activation_leg.svg'))
    plt.close()

# During which scenes is the full sample synchronized?

## Compute intersubject phase synchrony and a null distribution

In [ ]:
for movie in ['DM','TP']:
    for sample in ['cbic','rubic']:
        print(sample, movie)
        # set up folder
        out_folder = os.path.join(out_dir, 'fullsample', 'dynamic_{0}_movie{1}_mean'.format(sample, movie))
        os.makedirs(out_folder, exist_ok=True)
        
        # compute mean timeseries similarity across sample
        group_data_file = os.path.join(out_folder,'compiled_timeseries_data_{0}_movie{1}.npy'.format(sample, movie))
        group_data = np.load(group_data_file)
        print(group_data.shape)
        mask = np.tri(group_data.shape[2], group_data.shape[2], -1, dtype=int)
        isps_data = np.memmap(os.path.join(out_dir, 'maturity','dynamic_{0}_movie{1}_age'.format(sample, movie), 
                                           'isps_data.dat'), dtype=np.float32, mode='r', 
                              shape=(group_data.shape[1],group_data.shape[2],group_data.shape[2],group_data.shape[0]))
        isps_data = np.mean(isps_data[:,mask==1,:], axis=1).T
        ax0 = nib.cifti2.cifti2_axes.SeriesAxis(0,0.8,isps_data.shape[0], unit='second')
        img = nib.cifti2.cifti2.Cifti2Image(isps_data, (ax0, ax1))
        nib.save(img, os.path.join(out_folder, 'mean_dynamic_synchrony.ptseries.nii'))

        # make null distribution
        orig_shape = group_data.shape
        group_data = group_data.flatten()
        np.random.shuffle(group_data)
        group_data = group_data.reshape(orig_shape)
        phase_outfile = os.path.join(out_folder, 'permuted_phase_data_{0}_movie{1}.npy'.format(sample, movie))
        if os.path.isfile(phase_outfile):
            phase_data = np.load(phase_outfile)
        else:
            phase_data = compute_group_phase(group_data, phase_outfile)
        orig_shape = phase_data.shape
        outprefix = os.path.join(out_folder, 'permuted_sigonly_')
        _ , null_isps_data = compute_isps(phase_data, outprefix)
        null_isps_data = np.mean(null_isps_data[:,mask==1,:], axis=1)
        null_dist_file = os.path.join(out_folder, 'perm_sigonly_null_distribution.npy')
        np.save(null_dist_file, null_isps_data.flatten())

## Find the peaks in synchrony

In [19]:
for movie in ['TP','DM']:
    print(movie)
    if movie=='TP':
        video_dur = 200
    else:
        video_dur = 600
    ratings_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/processing/v1/summary/{0}_summary_codes_intuitivenames.csv'.format(movie)
    out_folder = os.path.join(out_dir, 'fullsample','dynamic_movie{0}_mean'.format(movie))
    os.makedirs(out_folder, exist_ok=True)

    # load discovery data
    disc_rho_file = os.path.join(out_dir, 'fullsample', 'dynamic_rubic_movie{0}_mean'.format(movie), 'mean_dynamic_synchrony.ptseries.nii')
    disc_null_file = os.path.join(out_dir, 'fullsample', 'dynamic_rubic_movie{0}_mean'.format(movie), 'perm_sigonly_null_distribution.npy')
    disc_mean_sim = nib.load(disc_rho_file).get_fdata()
    disc_null = np.sort(np.load(disc_null_file))

    # load replication data
    rep_rho_file = os.path.join(out_dir, 'fullsample', 'dynamic_cbic_movie{0}_mean'.format(movie), 'mean_dynamic_synchrony.ptseries.nii')
    rep_null_file = os.path.join(out_dir, 'fullsample', 'dynamic_cbic_movie{0}_mean'.format(movie), 'perm_sigonly_null_distribution.npy')
    rep_mean_sim = nib.load(rep_rho_file).get_fdata()
    rep_null = np.sort(np.load(rep_null_file))
    
    outprefix = os.path.join(out_folder, 'mean_bonncorr_')
    disc_df, rep_df = synchrony_discrep_fdr_netlevel(disc_mean_sim, disc_null, rep_mean_sim, outprefix, video_dur, alpha=0.05, 
                                                     TR=TR, parcel_labels=parcel_labels, atlas_file=atlas_file)
    disc_df.to_csv(os.path.join(out_folder, 'Discovery_mean_ISPS.csv'))
    rep_df.to_csv(os.path.join(out_folder, 'Replication_mean_ISPS.csv'))

TP
Disc sig threshold: 0.36739611625671387
Rep sig threshold: 0.368056982755661
no overlap between discovery and replication for Caudate
DM
Disc sig threshold: 0.36800816655158997
Rep sig threshold: 0.3687399923801422
no overlap between discovery and replication for Caudate


## Plot overall trends in synchrony

In [ ]:
for movie in ['DM','TP']:
    nois = ['Visual', 'Auditory', 'DorsalAttn', 'VentralAttn', 'CinguloOperc', 'Default', 'FrontoParietal'] 
    out_folder = os.path.join(out_dir, 'fullsample','dynamic_movie{0}_mean'.format(movie))
    disc_df = pd.read_csv(os.path.join(out_folder, 'Discovery_mean_ISPS.csv'), index_col=0)
    rep_df = pd.read_csv(os.path.join(out_folder, 'Replication_mean_ISPS.csv'), index_col=0)
    time = np.arange(0, len(disc_df)*0.8, 0.8)
    disc_df.index=np.round(time,1)
    rep_df.index=np.round(time,1)
    disc_df = disc_df.loc[:,nois]
    rep_df = rep_df.loc[:,nois]
            
    plt.figure(figsize=(12,10))
    sns.heatmap(disc_df.corr(), center=0)
    plt.xticks(rotation=30, ha='right')
    plt.tight_layout()
    plt.show()
    plt.close()
    plt.figure(figsize=(12,10))
    sns.heatmap(rep_df.corr(),center=0)
    plt.xticks(rotation=30, ha='right')
    plt.tight_layout()
    plt.show()
    plt.close()
    
    fig, ax = plt.subplots(len(disc_df.columns),1, sharex=True, figsize=(12, 1.5*len(disc_df.columns)))
    for i,net in enumerate(disc_df.columns):
        ax[i].plot(time, disc_df[net], color='k')
        ax[i].plot(time, rep_df[net], color='darkgray')
        ax[i].set_title(net)
        ax[i].set_xlim((0,time[-1]))
    sns.despine()
    plt.tight_layout()
    plt.show()
    plt.close()
    
    plt.figure(figsize=(12,4))
    sns.heatmap(disc_df.T,center=0.3, vmin=0.3, vmax=0.45)
    labels = [' ' for a in range(0, disc_df.shape[0])]
    if movie=='TP':
        x = [0, 100, 200]
    if movie=='DM':
        x = [0, 100, 200, 300, 400, 500, 600]   
    plt.xticks(x, x)
    plt.tight_layout()
    plt.show()
    plt.close()
    plt.figure(figsize=(12,4))
    sns.heatmap(rep_df.T,center=0.3, vmin=0.3, vmax=0.45)
    if movie=='TP':
        x = [0, 100, 200]
    if movie=='DM':
        x = [0, 100, 200, 300, 400, 500, 600]   
    plt.xticks(x, x)
    plt.tight_layout()
    plt.show()
    plt.close()
    
    disct = pd.melt(disc_df, value_vars = disc_df.columns, var_name='Network',value_name='Synchrony')
    plt.figure(figsize=(8,5))
    sns.violinplot(x='Network',y='Synchrony', data=disct)
    plt.axhline(y=0.368, color='k',linestyle='--')
    plt.xticks(rotation=30, ha='right')
    plt.title('Discovery')
    plt.ylim((0.35,0.5))
    plt.tight_layout()
    plt.show()
    plt.close()
    
    rept = pd.melt(rep_df, value_vars = disc_df.columns, var_name='Network',value_name='Synchrony')
    plt.figure(figsize=(8,5))
    sns.violinplot(x='Network',y='Synchrony', data=rept)
    plt.axhline(y=0.368, color='k',linestyle='--')
    plt.xticks(rotation=30, ha='right')
    plt.ylim((0.35,0.5))
    plt.title('Replication')
    plt.tight_layout()
    plt.show()
    plt.close()

# When are the oldest children synchronized?

## find synchrony peaks in oldest kids

In [6]:
## determine which scenes evoke greater synchrony across older children
nois = ['Visual', 'Auditory', 'DorsalAttn', 'VentralAttn', 'CinguloOperc', 'Default', 'FrontoParietal'] 

for movie in ['DM','TP']:
    for mat in ['age']:
        if movie=='DM':
            video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/Despicable_Me_1000.mp4'
            movie_dur=600
        else:
            video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/The_Present_0321.mp4'
            movie_dur = 200

        # identify oldest 20% of sample
        discmetric = subinfo.loc[(subinfo['site']=='rubic') & (subinfo['movie']==movie) & np.isfinite(subinfo[mat]),mat].to_numpy()
        discmask = (discmetric>=np.percentile(discmetric, 80)).astype(int)
        repmetric = subinfo.loc[(subinfo['site']=='cbic') & (subinfo['movie']==movie) & np.isfinite(subinfo[mat]),mat].to_numpy()
        repmask = (repmetric>=np.percentile(repmetric, 80)).astype(int)

        discovery_data = {'color':'#BAB3BF', 'matmask': {'mask': discmask, 'cutoff': np.percentile(discmetric, 80)}}
        replication_data = {'color':'#BAB3BF','matmask': {'mask': repmask, 'cutoff': np.percentile(repmetric, 80)}}
        both_data = {'color':'#BAB3BF','network_info':{}}
        time = np.arange(0,movie_dur, 0.8)

        ratings_file = os.path.join(project_dir, 'HBN_video_coding/processing/summary/{0}_summary_codes_intuitivenames.csv'.format(movie))
        out_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_top20'.format(movie, mat))
        os.makedirs(out_folder, exist_ok=True)

        # load discovery data
        disc_phase_file = os.path.join(out_dir, 'maturity', 'dynamic_rubic_movie{0}_{1}'.format(movie, mat), 
                                       'phase_data_rubic_movie{0}.npy'.format(movie))
        disc_phase = np.load(disc_phase_file)[:,:, discmask==1]
        out_prefix = os.path.join(out_folder, 'disc_oldest20_')
        if os.path.isfile(out_prefix + 'mean_isps_data.npy'):
            disc_mean_sim = np.load(out_prefix + 'mean_isps_data.npy')
        else:
            disc_mean_sim, _ = compute_isps(disc_phase, out_prefix)
        if disc_mean_sim.shape[1]!=len(parcel_labels):
            disc_mean_sim = disc_mean_sim.T

        # load replication data
        rep_phase_file = os.path.join(out_dir, 'maturity', 'dynamic_cbic_movie{0}_{1}'.format(movie, mat), 
                                      'phase_data_cbic_movie{0}.npy'.format(movie))
        rep_phase = np.load(rep_phase_file)[:,:, repmask==1]
        out_prefix = os.path.join(out_folder, 'rep_oldest20_')
        if os.path.isfile(out_prefix + 'mean_isps_data.npy'):
            rep_mean_sim = np.load(out_prefix + 'mean_isps_data.npy')
        else:
            rep_mean_sim, _ = compute_isps(rep_phase, out_prefix)
        if rep_mean_sim.shape[1]!=len(parcel_labels):
            rep_mean_sim = rep_mean_sim.T

        # identify sig parcels from sim analysis
        if mat=='PPS_score':
            sig_parcs_file = os.path.join(out_dir, 'maturity', 'ts_isc_movie{0}_{1}'.format(movie, 'puberty'),
                                          'movie{0}_isc_{1}_AnnaKmin_maskedrho_fdr0.01127.pscalar.nii'.format(movie, 'puberty'))
        else:
            sig_parcs_file = os.path.join(out_dir, 'maturity', 'ts_isc_movie{0}_{1}'.format(movie, mat),
                                          'movie{0}_isc_{1}_AnnaKmin_maskedrho_fdr0.01127.pscalar.nii'.format(movie, mat))
        sigparcsmask = (np.squeeze(nib.load(sig_parcs_file).get_fdata()) > 0).astype(int)

        # average across network/region
        disc_sim_net = np.zeros((disc_mean_sim.shape[0], len(nois)))
        rep_sim_net = np.zeros((rep_mean_sim.shape[0], len(nois)))

        for i, n in enumerate(nois):
            disc_sim_net[:,i] = np.mean(disc_mean_sim[:,(network_labels==n) & (sigparcsmask==1)], axis=1)
            rep_sim_net[:,i] = np.mean(rep_mean_sim[:,(network_labels==n) & (sigparcsmask==1)], axis=1)
            both_data['network_info'][n] = parcel_labels[(network_labels==n) & (sigparcsmask==1)]

            # make label file for sig parcels
            ax1 = nib.load(atlas_file).header.get_axis(1)
            data = nib.load(atlas_file).get_fdata()
            ax0 = nib.load(atlas_file).header.get_axis(0)
            newmap=dict()
            newmap[0] = ax0[0][1][0]
            parcels_keep = both_data['network_info'][n]
            for a in range(1,len(parcel_labels) +1):
                if parcel_labels[a-1] in parcels_keep:
                    newmap[a] = (parcel_labels[a-1], (142/255, 50/255, 209/255, 1))
                else:
                    newmap[a] = (parcel_labels[a-1], (1,1,1,0))
            ax0.label[0] = newmap
            img = nib.cifti2.cifti2.Cifti2Image(data, (ax0, ax1))
            nib.save(img, os.path.join(out_folder, 'sig_parcels_net-{0}.dlabel.nii'.format(n)))

        #### plot synchrony ####
        for i, net in enumerate(nois):
            fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
            # detect peaks in activation based on 2SDs above the mean
            discmean = np.mean(disc_sim_net[:,i])
            discmin = discmean + 1.5*np.std(disc_sim_net[:,i])
            dcpeaks, dcproperties = scs.find_peaks(disc_sim_net[:,i], width=5, height=discmin)

            repmean = np.mean(rep_sim_net[:,i])
            repmin = repmean + 1.5*np.std(rep_sim_net[:,i])
            rcpeaks, rcproperties = scs.find_peaks(rep_sim_net[:,i], width=5, height=repmin)

            peaks = pd.DataFrame(0,index=np.arange(0,movie_dur, TR), columns=['disc','rep'])
            peaks['time'] = np.arange(0,movie_dur, TR)
            for i_c, c in enumerate(dcproperties['prominences']):
                peaks.loc[dcproperties['left_ips'][i_c]*TR:dcproperties['right_ips'][i_c]*TR, 'disc'] = 1
            for i_c, c in enumerate(rcproperties['prominences']):
                peaks.loc[rcproperties['left_ips'][i_c]*TR:rcproperties['right_ips'][i_c]*TR, 'rep'] = 1
            peaks['disc_mean'] = disc_sim_net[:,i]
            peaks['rep_mean'] = rep_sim_net[:,i]
            both_data[net]={'long_peaks':peaks}
            peaks['both'] = ((peaks['disc']==1) & (peaks['rep']==1)).astype(int)
            peaks['segment'] = (peaks['both'].diff(1) != 0).astype(int).cumsum()
            res = pd.DataFrame({'start': peaks.groupby('segment').time.first(),
                                'end': peaks.groupby('segment').time.last(),
                                'dur': peaks.groupby('segment').time.last()-peaks.groupby('segment').time.first(),
                                'disc_mean':peaks.groupby('segment')['disc_mean'].mean(),
                                'rep_mean':peaks.groupby('segment')['rep_mean'].mean(),
                                'disc_max':peaks.groupby('segment')['disc_mean'].max(),
                                'rep_max':peaks.groupby('segment')['rep_mean'].max(),
                                'value': peaks.groupby('segment')['both'].mean()}).reset_index(drop=True)
            peaks_to_delete = res.loc[(res['value'] == 1) & (res['dur']<4), :]
            if len(peaks_to_delete)>0:
                for a in peaks_to_delete.index:
                    both_data[net]['long_peaks'].loc[peaks_to_delete.loc[a,'start']:peaks_to_delete.loc[a,'end'], 'both'] = 0
            peaks = res.loc[(res['value'] == 1) & (res['dur']>=4), :]

            for i_c, c in enumerate(peaks['start']):
                ax[0].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.9)
                ax[1].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.9)

            # plot traces
            ax[0].plot(time, disc_sim_net[:,i], color='k')
            ax[0].set_xlim((0,movie_dur))
            ax[0].set_title('Discovery')            
            ax[1].plot(time, rep_sim_net[:,i], color='k')
            ax[1].set_xlim((0,movie_dur))
            ax[1].set_title('Replication')

            both_data[net]['peaks'] = peaks
            plt.suptitle(net)
            sns.despine()
            plt.tight_layout()
            plt.savefig(os.path.join(out_folder, 'Network_bysite_synchrony_{0}.svg'.format(net)))
            plt.close()

        fig, ax = plt.subplots(len(nois),1,figsize=(12,2*len(nois)), sharex=True)
        for i, net in enumerate(nois):    
            ax[i].plot(time, disc_sim_net[:,i], color='k')
            ax[i].set_xlim((0,movie_dur))
            ax[i].set_title(net)

            peaks = both_data[net]['peaks']
            for i_c, c in enumerate(peaks['start']):
                ax[i].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.9)
                ax[i].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.9)

        sns.despine()
        plt.tight_layout()
        plt.savefig(os.path.join(out_folder, 'Networks_all_synchrony.svg'))
        plt.close()

        ### Find differences in peak versus non-peak in terms of video features
        for net in nois:
            peak_mask = both_data[net]['long_peaks']['both'].to_numpy()
            ratings_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/processing/v1/summary/{0}_summary_codes_intuitivenames.csv'.format(movie)
            out_file = os.path.join(out_folder, '{0}_peak_differences.svg'.format(net))
            if (peak_mask.max()==1):
                both_data[net]['peak_quant_analysis'] = find_peak_rating_diffs(peak_mask, ratings_file, movie_dur, out_file, color=both_data['color'], TR=TR, alpha=0.05, fdr=True, shift=6)

        ### find which clips are promoting the most activation
        for net in nois:
            peaktimes = both_data[net]['peaks']
            peaktimes.loc[:,'meanact'] = peaktimes.loc[:,['disc_mean','rep_mean']].mean(axis=1)
            peaktimes.loc[:,'maxact'] = peaktimes.loc[:,['disc_max','rep_max']].mean(axis=1)
            peaktimes = peaktimes.sort_values(by='maxact', ascending=False)
            peaktimes.index = range(1, len(peaktimes)+1)
            both_data[net]['peak_qual_analysis'] = peaktimes
            outfolder = os.path.join(out_folder, 'clips_{0}'.format(net))
            os.makedirs(outfolder, exist_ok=True)
            peaktimes.to_csv(os.path.join(outfolder, 'peak_analysis_{0}.csv'.format(net)))
            shift = 5 #in seconds
            ratings_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/processing/v1/summary/{0}_summary_codes10Hz_intuitivenames.csv'.format(movie)
            match_peak_to_clips(peaktimes, shift, video_file, outfolder, ratings_file, 
                                bool_list=['faces','body','closeup','spoken_words','written_words'],
                                dim_list=['positive','negative','brightness','loudness','saliency'])

        ## pickle results
        f = open(os.path.join(out_folder, 'Discovery_raw_signal_data.pkl'),'wb')
        pickle.dump(discovery_data,f)
        f = open(os.path.join(out_folder, 'Replication_raw_signal_data.pkl'),'wb')
        pickle.dump(replication_data,f)
        f = open(os.path.join(out_folder, 'final_peaks_data.pkl'),'wb')
        pickle.dump(both_data,f)

No differences between peak and nonpeak ratings
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


...

/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1667: SettingW

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()
/Users/catcamacho/Library/Python/3.8/lib/python/site-packages/pandas/plotting/_matplotlib/__init__.py:71: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  plot_obj.generate()


## plot oldest, youngest, and middle 20%

In [ ]:
mat = 'age'
nois = ['Visual', 'Auditory', 'DorsalAttn', 'VentralAttn', 'CinguloOperc', 'Default', 'FrontoParietal'] 

for movie in ['TP','DM']:
    if movie=='DM':
        video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/Despicable_Me_1000.mp4'
        movie_dur=600
    else:
        video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/The_Present_0321.mp4'
        movie_dur = 200

    # identify youngest 20% of sample
    discmetric = subinfo.loc[(subinfo['site']=='rubic') & (subinfo['movie']==movie) & np.isfinite(subinfo[mat]),mat].to_numpy()
    discmask = (discmetric<=np.percentile(discmetric, 20)).astype(int)
    repmetric = subinfo.loc[(subinfo['site']=='cbic') & (subinfo['movie']==movie) & np.isfinite(subinfo[mat]),mat].to_numpy()
    repmask = (repmetric<=np.percentile(repmetric, 20)).astype(int)

    out_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_bottom20'.format(movie, mat))
    os.makedirs(out_folder, exist_ok=True)

    # pull discovery data
    disc_phase_file = os.path.join(out_dir, 'maturity', 'dynamic_rubic_movie{0}_{1}'.format(movie, mat), 
                                   'phase_data_rubic_movie{0}.npy'.format(movie))
    disc_phase = np.load(disc_phase_file)[:,:, discmask==1]
    out_prefix = os.path.join(out_folder, 'disc_youngest20_')
    disc_mean_sim, _ = compute_isps(disc_phase, out_prefix)

    # pull replication data
    rep_phase_file = os.path.join(out_dir, 'maturity', 'dynamic_cbic_movie{0}_{1}'.format(movie, mat), 
                                  'phase_data_cbic_movie{0}.npy'.format(movie))
    rep_phase = np.load(rep_phase_file)[:,:, repmask==1]
    out_prefix = os.path.join(out_folder, 'rep_youngest20_')
    rep_mean_sim, _ = compute_isps(rep_phase, out_prefix)

In [ ]:
mat = 'age'
nois = ['Visual', 'Auditory', 'DorsalAttn', 'VentralAttn', 'CinguloOperc', 'Default', 'FrontoParietal'] 

for movie in ['TP','DM']:
    if movie=='DM':
        video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/Despicable_Me_1000.mp4'
        movie_dur=600
    else:
        video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/The_Present_0321.mp4'
        movie_dur = 200

    # identify middle 20% of sample
    discmetric = subinfo.loc[(subinfo['site']=='rubic') & (subinfo['movie']==movie) & np.isfinite(subinfo[mat]),mat].to_numpy()
    discmask = ((discmetric>=np.percentile(discmetric, 40)) & (discmetric<=np.percentile(discmetric, 60))).astype(int)
    repmetric = subinfo.loc[(subinfo['site']=='cbic') & (subinfo['movie']==movie) & np.isfinite(subinfo[mat]),mat].to_numpy()
    repmask = ((repmetric>=np.percentile(repmetric, 40)) & (repmetric<=np.percentile(repmetric, 60))).astype(int)

    out_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_middle20'.format(movie, mat))
    os.makedirs(out_folder, exist_ok=True)

    # pull discovery data
    disc_phase_file = os.path.join(out_dir, 'maturity', 'dynamic_rubic_movie{0}_{1}'.format(movie, mat), 
                                   'phase_data_rubic_movie{0}.npy'.format(movie))
    disc_phase = np.load(disc_phase_file)[:,:, discmask==1]
    out_prefix = os.path.join(out_folder, 'disc_middle20_')
    disc_mean_sim, _ = compute_isps(disc_phase, out_prefix)

    # pull replication data
    rep_phase_file = os.path.join(out_dir, 'maturity', 'dynamic_cbic_movie{0}_{1}'.format(movie, mat), 
                                  'phase_data_cbic_movie{0}.npy'.format(movie))
    rep_phase = np.load(rep_phase_file)[:,:, repmask==1]
    out_prefix = os.path.join(out_folder, 'rep_middle20_')
    rep_mean_sim, _ = compute_isps(rep_phase, out_prefix)

In [ ]:
for movie in ['TP','DM']:
    if movie=='TP':
        movie_dur = 200
    else:
        movie_dur = 600
    time = np.arange(0,movie_dur, 0.8)
    out_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_both'.format(movie, mat))
    os.makedirs(out_folder, exist_ok=True)
    
    # load data from oldest 20% of sample
    top_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_top20'.format(movie, mat))
    top_peaks = pickle.load(open(os.path.join(top_folder, 'final_peaks_data.pkl'),'rb'))
    top_disc_mean = np.load(os.path.join(top_folder, 'disc_oldest20_mean_isps_data.npy'))
    top_rep_mean = np.load(os.path.join(top_folder, 'rep_oldest20_mean_isps_data.npy'))
    
    # load data from middle 20% of sample
    mid_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_middle20'.format(movie, mat))
    mid_disc_mean = np.load(os.path.join(mid_folder, 'disc_middle20_mean_isps_data.npy'))
    mid_rep_mean = np.load(os.path.join(mid_folder, 'rep_middle20_mean_isps_data.npy'))

    # load data from youngest 20% of sample
    bot_folder = os.path.join(out_dir, 'maturity','dynamic_movie{0}_{1}_bottom20'.format(movie, mat))
    bot_disc_mean = np.load(os.path.join(bot_folder, 'disc_youngest20_mean_isps_data.npy'))
    bot_rep_mean = np.load(os.path.join(bot_folder, 'rep_youngest20_mean_isps_data.npy'))

    for i, n in enumerate(nois):
        # plot disc and replication
        fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
        for i_c, c in enumerate(top_peaks[n]['peaks'].index):
            ax[0].axvspan(top_peaks[n]['peaks'].loc[c, 'start'], top_peaks[n]['peaks'].loc[c, 'end'], color='#B9B4BD',fill=True, alpha=0.9)
            ax[1].axvspan(top_peaks[n]['peaks'].loc[c, 'start'], top_peaks[n]['peaks'].loc[c, 'end'], color='#B9B4BD',fill=True, alpha=0.9)

        # plot traces
        ax[0].plot(time, np.mean(top_disc_mean[:,network_labels==n], axis=1), color='#9B59DA')
        ax[0].plot(time, np.mean(mid_disc_mean[:,network_labels==n], axis=1), color='#62388a')
        ax[0].plot(time, np.mean(bot_disc_mean[:,network_labels==n], axis=1), color='k')
        ax[0].set_xlim((0,movie_dur))
        ax[0].set_title('Discovery')            
        ax[1].plot(time, np.mean(top_rep_mean[:,network_labels==n], axis=1), color='#9B59DA')
        ax[1].plot(time, np.mean(mid_rep_mean[:,network_labels==n], axis=1), color='#62388a')
        ax[1].plot(time, np.mean(bot_rep_mean[:,network_labels==n], axis=1), color='k')
        ax[1].set_xlim((0,movie_dur))
        ax[1].set_title('Replication')
        plt.suptitle(n)
        sns.despine()
        plt.tight_layout()
        plt.savefig(os.path.join(out_folder, 'Network_bysite_synchrony_{0}.svg'.format(n)))
        plt.show()
        plt.close()

    fig, ax = plt.subplots(len(nois),1,figsize=(12,2*len(nois)), sharex=True)
    for i, n in enumerate(nois):    
        ax[i].plot(time, np.mean(top_disc_mean[:,network_labels==n], axis=1), color='#9B59DA')
        ax[i].plot(time, np.mean(mid_disc_mean[:,network_labels==n], axis=1), color='#62388a')
        ax[i].plot(time, np.mean(bot_disc_mean[:,network_labels==n], axis=1), color='k')
        ax[i].set_xlim((0,movie_dur))
        ax[i].set_title(n)

        for i_c, c in enumerate(top_peaks[n]['peaks'].index):
            ax[i].axvspan(top_peaks[n]['peaks'].loc[c, 'start'], top_peaks[n]['peaks'].loc[c, 'end'], color='#B9B4BD',fill=True, alpha=0.9)

    sns.despine()
    plt.tight_layout()
    plt.savefig(os.path.join(out_folder, 'Networks_all_synchrony.svg'))
    plt.show()
    plt.close()